In [38]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
import json
import re

In [130]:
class question_matcher:
    def __init__(self):
        self.patterns= []
        self.q_json = self.get_questions_json()
        self.matcher = Matcher(nlp.vocab)

    def get_questions_json(self):
        with open('questions.json', 'r') as f:
            pat = json.load(f)
        return pat
    
    def define_keyword(self,question:str):
        for q in self.q_json:
            if q['keyword'] in question.lower():
                keyword = q['keyword']
                index = q['qid']
        return keyword, index

    def get_focus(self,question:str,ind,pattern):
        focus =''
        focus_pos = pattern['focus']
        token_q = [token.text for token in nlp(question)]
        i = 0
        start = 0
        for token in focus_pos[ind].split():
            if(token == 'sf'):
                start = 1
                continue
                
            if(token == 'ef'):
                start = 0
                break;
            if(start == 1):
                focus = focus + ' ' + token_q[i]
            i = i + 1
        return focus
    
    def re_get_focus(self,question:str,ind,pattern):
        question_token = [token.text for token in nlp(question)]
        focus = ''
        focus_pos = pattern['focus']
        start = 0
        i = 0
        for token in focus_pos[ind].split('\s*'):
                if(token == 'sf'):
                    start = 1
                    i = i - 1
                    continue

                if(token == 'ef'):
                    start = 0
                    break

                if(token == ".*" and start == 1):
                    j = i
                    while(j < len(question_token)):
                        focus = focus + ' ' + question_token[j]
                        j = j + 1
                    focus = focus.strip()
                    print(focus)
                    return focus

                if(start == 1):
                    focus = focus + ' ' + question_token[i]
                
                i = i + 1

                # if(i == len(question_token)):
                #     print(i)
                #     break

        focus = focus.strip()
        print(focus)

    def match_pattern(self,question):
        q_pos = [token.pos_ for token in nlp(question)]
        keyword, index = self.define_keyword(question)
        for pattern in self.q_json[index]['patterns']:
            pos = self.pattern_to_pos(pattern['pos'])
            self.matcher.add(p, pos)
            doc = nlp(question)
            matches = self.matcher(doc)
            if matches != []:
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]
                    span = doc[start:end]
                    print(string_id)
                    print(q_pos)
                    print(pattern['pos'])
                    print(match_id)
            else:
                print('fail')

    def re_match_pattern(self,question):
        found = 0
        doc = nlp(question)
        ltoken = [token.pos_ for token in nlp(question)]
        q_pattern = ''
        for token in doc:
            q_pattern = q_pattern + token.pos_ + ' ' 
        keyword, index = self.define_keyword(question)
        for pattern in self.q_json[index]['patterns']:
            matched_pos=0
            for pos in pattern['pos']:
                regexpos  = f'({pos})'
                matches = re.fullmatch(regexpos, q_pattern)
                if matches != None:
                    print('Matched!')
                    print(matches.string)
                    print(pos)
                    print(pattern['pattern'])
                    self.re_get_focus(question,matched_pos,pattern)
                    found = 1
                    break
                matched_pos = matched_pos + 1
            if found == 1:
                break
            


    def on_match(self,matcher, doc, id, matches):
        print('Matched!', matches)


    def pattern_to_pos(self, patterns:list):
        pos = []
        pattern = []
        for p in patterns:
            
            itr = []
            pattern.append(p)
            for token in p.split():
                if token != '*':
                    itr.append({'POS':token})
                else:
                    itr.append({})
            pos.append(itr)
        return pos
qm = question_matcher()


In [131]:
qm.re_match_pattern('what is atom')

Matched!
PRON AUX PROPN 
\s*PRON\s*AUX\s*PROPN\s*
what is [ENTITY]
atom


In [102]:
with open('questions.json', 'r') as f:
    patt = json.load(f)



In [127]:
doc = nlp("what is atom")
print( [token.pos_ for token in doc])
print( [token.text for token in doc])

['PRON', 'AUX', 'PROPN']
['what', 'is', 'atom']


In [124]:
doc = nlp("when did Bill Gates graduate college")
print( [token.pos_ for token in doc])
print( [token.text for token in doc])

['SCONJ', 'VERB', 'PROPN', 'VERB', 'NOUN']
['when', 'did', 'Bill Gates', 'graduate', 'college']


In [47]:
pos = '.'
patteee = f'({".*"})'
matches = re.fullmatch(patteee, "llllllll   ll")
print(matches)

<re.Match object; span=(0, 8), match='llllllll'>
